In [2]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import random
import datetime

import tensorflow as tf
import numpy as np
import scipy.misc
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data
slim = tf.contrib.slim

In [3]:
mnist = input_data.read_data_sets('MNIST_data')
x_size, y_size = 28, 28
n_classes = 10
default_collection = 'nodes'

def timestamp():
    d = datetime.datetime.now()
    return d.strftime("%Y/%m/%d/%X")

timestamp()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


'2016/12/11/00:19:38'

In [4]:
def random_batch_iterator(x, y, batch_size):
    n = x.shape[0]
    assert n == y.shape[0]
    
    while True:
        index = np.random.randint(n, size=batch_size)
        x_batch, y_batch = x[index], y[index]
        yield x_batch.copy(), y_batch.copy()
        
def batch_iterator(x, y, batch_size):
    n = x.shape[0]
    assert n == y.shape[0]
    
    for i in range(0, n, batch_size):
        x_batch, y_batch = x[i:i+batch_size], y[i:i+batch_size]        
        yield x_batch.copy(), y_batch.copy()

In [5]:
mnist.train.images.shape

(55000, 784)

In [6]:
def build_cnn(inputs, n_conv, conv_base, conv_mul, conv_size, pool_size, collection=default_collection):
    l = inputs
    for i in range(n_conv):
        n_filters = conv_base * conv_mul ** i
        l = slim.conv2d(l, n_filters, [conv_size, conv_size],
                        scope='Conv{}'.format(i+1))
        l = slim.max_pool2d(l, [pool_size, pool_size], scope='MaxPool{}'.format(i+1))
    l = slim.flatten(l)
    
    l = slim.dropout(l, 0.5, scope='Dropout', outputs_collections=collection)
    l = slim.fully_connected(l, 10, activation_fn=None, scope='Output',
                             outputs_collections=collection)
    return l

def build_loss(logits, y_true):
    logloss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_true))
    return logloss

In [7]:
batch_size = 512

n_conv = 2
conv_base = 32
conv_mul = 2
conv_size = 5
pool_size = 2

#graph = tf.Graph()
#with graph.as_default():
x_ph = tf.placeholder(tf.float32, shape=[batch_size, x_size * y_size])
x_image = tf.reshape(x_ph, [-1, x_size, y_size, 1])
y_ph = tf.placeholder(tf.int64, shape=[batch_size])
logits = build_cnn(x_image, n_conv=n_conv, conv_base=conv_base, conv_mul=conv_mul,
                           conv_size=conv_size, pool_size=pool_size)

prediction = tf.nn.softmax(logits)

loss = build_loss(logits, y_ph)

optimizer = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.argmax(prediction, 1), y_ph)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    # Code to use of tensorboard
    #with tf.name_scope('summaries'):
    #    tf.scalar_summary('log_loss', loss)
    #    tf.scalar_summary('acc', accuracy)
    #    merged_summary = tf.merge_all_summaries()

#nodes = graph.get_collection(default_collection)

In [8]:
mnist.train.images

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [12]:
n_epochs = 1000

train_iterator = random_batch_iterator(mnist.train.images, mnist.train.labels, batch_size=batch_size)
val_iterator = random_batch_iterator(mnist.validation.images, mnist.validation.labels, batch_size=batch_size)
test_iterator = batch_iterator(mnist.test.images, mnist.test.labels, batch_size=batch_size)

best_acc = 0.0
path = '/tmp/tf/' + timestamp()
#variables_to_restore = slim.get_variables_by_name("Conv")
#tf.train.import_meta_graph('my-model.meta')
#variables_to_restore = slim.get_model_variables()
variables_to_restore = slim.get_variables(scope="Conv1")
print(variables_to_restore)
#restore = tf.train.import_meta_graph('Mnist_NLA_2.ckpt.meta')
#print(slim.get_model_variables())
print(tf.train.latest_checkpoint('./'))
restore = tf.train.Saver(variables_to_restore)
with tf.Session() as session:
    restore.restore(session, tf.train.latest_checkpoint('./'))
    conv1 = slim.get_variables(scope="Conv1")
    conv1_arr = conv1[0].eval()
    print(conv1_arr.shape)

[<tensorflow.python.ops.variables.Variable object at 0x7f0b3a5dfb10>, <tensorflow.python.ops.variables.Variable object at 0x7f0b3a5dfbd0>, <tensorflow.python.ops.variables.Variable object at 0x7f0b3a3f5d10>, <tensorflow.python.ops.variables.Variable object at 0x7f0b3a443cd0>, <tensorflow.python.ops.variables.Variable object at 0x7f0b3a4062d0>, <tensorflow.python.ops.variables.Variable object at 0x7f0b3a406490>]
./Mnist_NLA_2_trial.ckpt
(5, 5, 1, 32)


In [ ]:
test_acc